<a href="https://colab.research.google.com/github/jess22jess/EDP/blob/main/Membrana_vibratoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# Parámetros físicos
c = 1.0  # Velocidad de propagación de la onda
Lx = 4.0  # Longitud en dirección x (0-4)
Ly = 2.0  # Longitud en dirección y (0-2)

# Condición inicial de desplazamiento
def f(x, y):
    return 0.1 * (4*x - x**2) * (2*y - y**2)

# Condición inicial de velocidad
def g(x, y):
    return 0.0

# Número de términos en la serie de Fourier
N = 20
M = 20

# Precalcular los coeficientes de Fourier A_mn y B_mn
def calcular_coeficientes(N, M):
    A = np.zeros((N, M))
    B = np.zeros((N, M))

    # Integración numérica para calcular los coeficientes
    n_points = 100  # Puntos para la integración numérica

    x = np.linspace(0, Lx, n_points)
    y = np.linspace(0, Ly, n_points)
    X, Y = np.meshgrid(x, y)
    dx = Lx / (n_points - 1)
    dy = Ly / (n_points - 1)

    for m in range(1, N+1):
        for n in range(1, M+1):
            # Coeficiente A_mn
            integrando_A = f(X, Y) * np.sin(m*np.pi*X/Lx) * np.sin(n*np.pi*Y/Ly)
            A[m-1, n-1] = (4/(Lx*Ly)) * np.sum(integrando_A) * dx * dy

            # Coeficiente B_mn
            integrando_B = g(X, Y) * np.sin(m*np.pi*X/Lx) * np.sin(n*np.pi*Y/Ly)
            B[m-1, n-1] = (4/(Lx*Ly)) * np.sum(integrando_B) * dx * dy / (c * np.sqrt((m*np.pi/Lx)**2 + (n*np.pi/Ly)**2))

    return A, B

A, B = calcular_coeficientes(N, M)

# Función de solución u(x,y,t)
def u(x, y, t):
    total = 0.0
    for m in range(1, N+1):
        for n in range(1, M+1):
            λ_mn = c * np.sqrt((m*np.pi/Lx)**2 + (n*np.pi/Ly)**2)
            total += (A[m-1, n-1] * np.cos(λ_mn * t) + B[m-1, n-1] * np.sin(λ_mn * t)) * \
                     np.sin(m*np.pi*x/Lx) * np.sin(n*np.pi*y/Ly)
    return total

# Malla para la visualización
x = np.linspace(0, Lx, 50)
y = np.linspace(0, Ly, 50)
X, Y = np.meshgrid(x, y)

# Figura 3D
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Función para inicializar la animación
def init():
    Z = u(X, Y, 0)
    surf = ax.plot_surface(X, Y, Z, cmap='viridis')
    ax.set_xlabel('Posición X (ft)')
    ax.set_ylabel('Posición Y (ft)')
    ax.set_zlabel('Desplazamiento (ft)')
    ax.set_title('Vibración de una Membrana Rectangular')
    ax.set_zlim(-0.5, 0.5)
    return surf,

# Función para actualizar la animación
def update(frame):
    ax.clear()
    Z = u(X, Y, frame*0.1)  # Paso de tiempo de 0.1
    surf = ax.plot_surface(X, Y, Z, cmap='viridis')
    ax.set_xlabel('Posición X (ft)')
    ax.set_ylabel('Posición Y (ft)')
    ax.set_zlabel('Desplazamiento (ft)')
    ax.set_title(f'Vibración de una Membrana Rectangular (t = {frame*0.1:.1f} s)')
    ax.set_zlim(-0.5, 0.5)
    return surf,

# Crear animación
ani = FuncAnimation(fig, update, frames=100, init_func=init, blit=False, interval=50)

# Mostrar la animación
plt.close()
HTML(ani.to_html5_video())